**Observed Trends**
* The majority of customers are male at 81% of total users.
* Over 40% of all transactions are driven by young adult users aged 20-24.
* Items "Final Critic", "Retribution Axe" and "Stormcaller" are in the top 5 most popular items and are also top revenue drivers.

In [227]:
import pandas as pd

In [228]:
#Import file
file = pd.read_json('purchase_data.json')
file.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [229]:
#Code to take a peek at second file
Alt_file = pd.read_json('purchase_data2.json')

In [230]:
#Count the number of unique players
Number_Players = file["SN"].unique().size
#Purchasing Analysis: number of unique items
Number_Items = file["Item Name"].unique().size
#Calculate avg purchase price
Avg_Price = file["Price"].mean()
#Count the number of unique purchases
Number_Purchases = file["Item ID"].size
#Add revenue
Revenue = file["Price"].sum()
print(f" Number_of_players: {Number_Players}\n Number_of_items: {Number_Items}\n Average_Price: {Avg_Price}\n Number_of_Purchases: {Number_Purchases}\n Total_Revenue: {Revenue}")

 Number_of_players: 573
 Number_of_items: 179
 Average_Price: 2.9311923076923074
 Number_of_Purchases: 780
 Total_Revenue: 2286.33


In [231]:
#Create table to remove user duplicates and find user gender
Gender_Table = pd.DataFrame(file.groupby('SN')['Gender'].unique())
Gender_Table.reset_index(inplace=True)
Gender_Table.columns=["SN","Gender"]
#Count of customers by gender
Gender_count = pd.DataFrame(Gender_Table['Gender'].value_counts())
Gender_count.reset_index(inplace=True)
Gender_count.columns=["Gender","Count"]
Gender_count["Percent of Total"]=(Gender_count["Count"]/Number_Players)*100
Gender_count

,Gender,Count,Percent of Total
0,[Male],465,81.151832
1,[Female],100,17.452007
2,[Other / Non-Disclosed],8,1.396161


In [232]:
#Purchasing Analysis (Gender)
Count_by_Gender = pd.DataFrame(file.groupby('Gender')['SN'].count())
Count_by_Gender.reset_index(inplace=True)
Count_by_Gender.columns=["Gender","Purchase Count"]
#Average Purchase Price
Avg_by_Gender = pd.DataFrame(file.groupby('Gender')['Price'].mean())
Avg_by_Gender.reset_index(inplace=True)
Avg_by_Gender.columns=["Gender","Average Price"]
#Total Purchase Value
Ttl_by_Gender = pd.DataFrame(file.groupby("Gender")["Price"].sum())
Ttl_by_Gender.reset_index(inplace=True)
Ttl_by_Gender.columns=["Gender","Total Purchase"]
#Normalized values
Normalized_gender = (pd.DataFrame(file.groupby("Gender")["Price"].sum()))/Gender_count['Count'].sum()
Normalized_gender.reset_index(inplace=True)
Normalized_gender.columns=["Gender","Normalized Values"]
# Create a new table consolidating above calculations
merge_table1 = pd.merge(Count_by_Gender, Avg_by_Gender, on="Gender")
merge_table2 = pd.merge(merge_table1, Ttl_by_Gender, on="Gender")
merge_table = pd.merge(merge_table2, Normalized_gender, on="Gender")
merge_table

,Gender,Purchase Count,Average Price,Total Purchase,Normalized Values
0,Female,136,2.815515,382.91,0.668255
1,Male,633,2.950521,1867.68,3.259476
2,Other / Non-Disclosed,11,3.249091,35.74,0.062373


In [233]:
Age_Table = pd.DataFrame(file.groupby('SN')['Age'].unique())
Age_Table.reset_index(inplace=True)
Age_Table.columns=["SN","Age"]
#Age_Table.head()

In [234]:
#Analysis by age groups: Bins are in 4 year increments (Example: <10, 10-14, 15-19, etc.)
Bins = [0,9,14,19,24,29,34,39,44,49]
Group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49']
group_file= pd.cut(file["Age"], Bins, labels=Group_names)
#group_file.head()

In [235]:
#Add bins to file
file["Age_Group"] = pd.cut(file["Age"], Bins, labels=Group_names)
file.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age_Group
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [236]:
#file2.reset_index(inplace=True)
file.columns=["Age","Gender","Item ID","Item Name","Price","SN","Age_Group"]
file.reset_index(inplace=True)

In [237]:
#Analysis by Age
#Purchase count by age group
Count_by_Age = pd.DataFrame(file.groupby('Age_Group')['SN'].count())
Count_by_Age.reset_index(inplace=True)
Count_by_Age.columns=["Age_Group","Purchase Count"]
#Average Purchase Price
Avg_by_Age = pd.DataFrame(file.groupby('Age_Group')['Price'].mean())
Avg_by_Age.reset_index(inplace=True)
Avg_by_Age.columns=["Age_Group","Average Price"]
#Total Purchase Value
Ttl_by_Age = pd.DataFrame(file.groupby("Age_Group")["Price"].sum())
Ttl_by_Age.reset_index(inplace=True)
Ttl_by_Age.columns=["Age_Group","Total Purchase"]
#Normalized values
Normalized_Age = (pd.DataFrame(file.groupby("Age_Group")["SN"].count()))/Number_Purchases
Normalized_Age.reset_index(inplace=True)
Normalized_Age.columns=["Age_Group","Normalized Values"]
# Create a new table consolidating above calculations
age_merge_table1 = pd.merge(Count_by_Age, Avg_by_Age, on="Age_Group")
age_merge_table2 = pd.merge(age_merge_table1, Ttl_by_Age, on="Age_Group")
age_merge_table = pd.merge(age_merge_table2, Normalized_Age, on="Age_Group")
age_merge_table

,Age_Group,Purchase Count,Average Price,Total Purchase,Normalized Values
0,<10,28,2.980714,83.46,0.035897
1,10-14,35,2.770000,96.95,0.044872
2,15-19,133,2.905414,386.42,0.170513
3,20-24,336,2.913006,978.77,0.430769
4,25-29,125,2.962640,370.33,0.160256
5,30-34,64,3.082031,197.25,0.082051
6,35-39,42,2.842857,119.40,0.053846
7,40-44,16,3.189375,51.03,0.020513
8,45-49,1,2.720000,2.72,0.001282


In [238]:
#Top Spenders
Count_by_User = pd.DataFrame(file.groupby('SN')['Item ID'].count())
Count_by_User.reset_index(inplace=True)
Count_by_User.columns=["SN","Purchase Count"]
#Count_by_User.head()
#Avg Price
Avg_by_User = pd.DataFrame(file.groupby('SN')['Price'].mean())
Avg_by_User.reset_index(inplace=True)
Avg_by_User.columns=["SN","Average Price"]
#Total Purchase Value
Ttl_by_User = pd.DataFrame(file.groupby("SN")["Price"].sum())
Ttl_by_User.reset_index(inplace=True)
Ttl_by_User.columns=["SN","Total Purchase"]
#Ttl_by_User
# Create a new table consolidating above calculations
user_merge_table1 = pd.merge(Count_by_User, Avg_by_User, on="SN")
user_merge_table = pd.merge(user_merge_table1, Ttl_by_User, on="SN")
#user_merge_table.head()

In [239]:
#Show top 5 spenders
user_merge_table.nlargest(5,'Total Purchase')

,SN,Purchase Count,Average Price,Total Purchase
538,Undirrala66,5,3.412000,17.06
428,Saedue76,4,3.390000,13.56
354,Mindimnya67,4,3.185000,12.74
181,Haellysu29,3,4.243333,12.73
120,Eoda93,3,3.860000,11.58


In [240]:
#Item ID and Item Name Key
ID_key = pd.DataFrame(file.groupby('Item Name')['Item ID'].unique())
ID_key.reset_index(inplace=True)
ID_key.columns=["Item Name","Item ID"]
#ID_key.head()

In [241]:
#Top Items: 
Count_by_Item = pd.DataFrame(file.groupby('Item Name')['Item ID'].count())
Count_by_Item.reset_index(inplace=True)
Count_by_Item.columns=["Item Name","Purchase Count"]
#Count_by_User.head()
#Avg Price
Avg_by_Item = pd.DataFrame(file.groupby('Item Name')['Price'].mean())
Avg_by_Item.reset_index(inplace=True)
Avg_by_Item.columns=["Item Name","Average Price"]
#Total Purchase Value
Ttl_by_Item = pd.DataFrame(file.groupby("Item Name")["Price"].sum())
Ttl_by_Item.reset_index(inplace=True)
Ttl_by_Item.columns=["Item Name","Total Purchase"]
#Ttl_by_User
# Create a new table consolidating above calculations
item_merge_table1 = pd.merge(Count_by_Item, Avg_by_Item, on="Item Name")
item_merge_table2 = pd.merge(item_merge_table1, ID_key, on="Item Name")
item_merge_table = pd.merge(item_merge_table2, Ttl_by_Item, on="Item Name")
item_merge_table = item_merge_table[['Item ID','Item Name', 'Purchase Count', 'Average Price', 'Total Purchase']]
#item_merge_table.head()

In [242]:
#Show top 5 items by purchase count
item_merge_table.nlargest(5,'Purchase Count')

,Item ID,Item Name,Purchase Count,Average Price,Total Purchase
56,"[92, 101]",Final Critic,14,2.757143,38.60
8,[84],Arcane Gem,11,2.230000,24.53
11,[39],"Betrayal, Whisper of Grieving Widows",11,2.350000,25.85
137,"[30, 180]",Stormcaller,10,3.465000,34.65
112,[34],Retribution Axe,9,4.140000,37.26


In [243]:
#Show top 5 items by sales volume
item_merge_table.nlargest(5,'Total Purchase')

,Item ID,Item Name,Purchase Count,Average Price,Total Purchase
56,"[92, 101]",Final Critic,14,2.757143,38.60
112,[34],Retribution Axe,9,4.140000,37.26
137,"[30, 180]",Stormcaller,10,3.465000,34.65
132,[115],Spectral Diamond Doomblade,7,4.250000,29.75
96,[32],Orenmir,6,4.950000,29.70
